In [ ]:
import pandas as pd
import numpy as np
import rasterio
import os
import pyreadr
import libpysal as ps
from esda.moran import Moran

Function for generating a grid of *StdBin* values as a numpy array from a dataframe of voxel hit data:

In [ ]:
def get_Std_arr(dataframe):
    dataframe = dataframe.assign(VAI=0)
    columns = dataframe.groupby(['X', 'Y'])
    std_df = pd.DataFrame(columns=["X", "Y", "Std_Col"]) # store StdBin values

    # Compute std of mean column leaf height for each column
    for (x, y), group in columns:
        StdBin, H_x = 0, 0 # mean leaf height and std of mean column leaf height
        for idx in group.index:
            group['VAI'][idx] = group['N'][idx] / sum(group['N'])
            H_x += group['Z'][idx]*group['VAI'][idx]
        for idx in group.index:
            var = (group['Z'][idx] - H_x)**2
            StdBin += group['VAI'][idx]*var # StdBinx from forestR
        
        new_row = {'X': x, 'Y': y, 'Std_Col': StdBin}
        std_df = std_df._append(new_row, ignore_index=True)
    
    pivot_df = std_df.pivot(index='Y', columns='X', values='Std_Col')
    # Convert the pivot DataFrame to a 2D NumPy array
    std_arr = pivot_df.to_numpy()   
    std_arr = np.nan_to_num(std_arr, nan=0.0) # set all nans to 0's

    return std_arr

Function for computing all three variants of *Canopy Rugosity*:

In [ ]:
def Compute_Vars(dir_path, res):

    files = [file for file in os.listdir(dir_path) if file.endswith(f"{res}m.RData")]
    res_df = pd.DataFrame(columns=['Plot', f'CanRug_{res}m_v1', f'CanRug_{res}m_v2', f'CanRug_{res}m_MoranI'])
   
    for i, f_name in enumerate(files):
        r_data = os.path.join(dir_path, f_name)
        # Read the RData file containg voxel hits into a dataframe
        vox_hits = pyreadr.read_r(r_data)
        df = vox_hits["vox_hits"] 
        res_df.at[i, 'Plot'] = f_name[6:8] # append plot number
        array = get_Std_arr(df) # get array of StdBin values at each (X,Y) location

        # v1: Overall variance of 2D grid
        res_v1 = np.std(array)
        res_df.at[i, f'CanRug_{res}m_v1'] = res_v1

        # v2: average of means of stds along X and Y 
        std_x = np.std(array, axis=1)
        std_y = np.std(array, axis=0)
        mean_x = np.mean(std_x)
        mean_y = np.mean(std_y)
        res_v2 = (mean_x + mean_y)/2
        res_df.at[i, f'CanRug_{res}m_v2'] = res_v2

        # Compute Moran's I
        weights = ps.weights.lat2W(array.shape[1], array.shape[0], rook=False)
        moran = Moran(array.flatten(), weights)
        res_df.at[i, f'CanRug_{res}m_MoranI'] = moran.I

    return res_df